# SET UP

In [ ]:
# Divine beast bless no bug here! 
#         ┌─┐    ┌─┐
#      ┌─┘ ┴───┘ ┴──┐
#      │                   │
#      │       ───       │
#      │  ─┬┘     └┬─  │
#      │                   │
#      │       ─┴─       │
#      │                   │
#      └─┐         ┌───┘
#          │         │
#          │         │
#          │         │
#          │         └──────────────┐
#          │                                  │
#          │                                  ├─┐
#          │                                  ┌─┘
#          │                                  │
#          └─┐  ┐  ┌──────┬──┐  ┌──┘
#            │  ─┤ ─┤         │  ─┤ ─┤
#            └──┴──┘         └──┴──┘

!pip install transformers
!pip install boto3
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..  


     |████████████████████████████████| 2.3MB 6.6MB/s 
     |████████████████████████████████| 901kB 27.1MB/s 
     |████████████████████████████████| 3.3MB 34.6MB/s 
     |████████████████████████████████| 133kB 6.4MB/s 
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 7.6MB 8.2MB/s 
     |████████████████████████████████| 143kB 48.7MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 8048 (delta 66), reused 65 (delta 28), pack-reused 7913
Receiving ob

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# DATA Preparation

In [ ]:
# data preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import numpy as np
import json
import h5py

# control
import argparse
import logging
from tqdm import trange

# pytorch
import torch
print(torch.__version__)
import torch.nn.functional as F

# transformers
from transformers import GPT2Config
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import BartForCausalLM, BartTokenizer,BartConfig

1.8.1+cu101


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
# the original dataframe
df_ori = pd.read_csv("./gdrive/MyDrive/COMP0087/full_dataset.csv")

In [ ]:
np.random.seed(10)
train_len = 500000
test_len = 200
subset_idx = np.random.choice(range(len(df_ori)),train_len+test_len,replace=False)
train_idx = subset_idx[0:train_len]
test_idx = subset_idx[train_len:]
df = df_ori.loc[subset_idx ,:]
# df_test = df_ori.loc[subset_idx[-100:],:]

In [ ]:
df.drop(df[df.title.map(lambda x: len(x)<4)].index, inplace=True)
df.drop(df[df.ingredients.map(lambda x: len(x)<2)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)].index, inplace=True)

In [ ]:
df.reset_index(inplace=True)
train, test = train_test_split(df, test_size=0.05) #use 5% for test set
# we only want first 10000 and 100 for train/test
# this has an error: train[75400:75600]
train = train[50000:400000]
test = test[0:100]

In [ ]:
def df_to_plaintext_file(input_df, output_file):
    print("Writing to", output_file)
    with open(output_file, 'w') as f:
        for index, row in input_df.iterrows():
            if index%100000==0:
                print(index)
            if type(row.NER)!=str:
                continue
            title = row.title
            directions = json.loads(row.directions)
            ingredients = json.loads(row.ingredients)
            ner = json.loads(row.NER)
            res = "<RECIPE_START> <INPUT_START> " + " <NEXT_INPUT> ".join(ner) + " <INPUT_END> <INGR_START> " + \
              " <NEXT_INGR> ".join(ingredients) + " <INGR_END> <INSTR_START> " + \
              " <NEXT_INSTR> ".join(directions) + " <INSTR_END> <TITLE_START> " + title + " <TITLE_END> <RECIPE_END>"
            f.write("{}\n".format(res))

In [ ]:
df_to_plaintext_file(train, 'unsupervised_train.txt')
df_to_plaintext_file(test, 'unsupervised_test.txt')

Writing to unsupervised_train.txt
300000
0
100000
400000
200000
Writing to unsupervised_test.txt


In [ ]:
#tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
special_tokens = {
    "additional_special_tokens": [
        "<TITLE_START>",
        "<TITLE_END>",
        "<INSTR_START>",
        "<NEXT_INSTR>",
        "<INSTR_END>",
        "<INGR_START>",
        "<NEXT_INGR>",
        "<INGR_END>",
        "<RECIPE_START>",
        "<RECIPE_END>",
        "<INPUT_START>",
        "<INPUT_END>",
        "<NEXT_INPUT>"
    ]
}

tokenizer.add_special_tokens(special_tokens)

end_token_id = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]

hf = h5py.File("unsupervised.h5", "w")
for filename in ["test", "train"]:
    out_np = []
    data = open("unsupervised_"+filename+".txt", "r")
    num = 0
    rows = 0
    last=[]
    for line in data:
        num+=1
        if num%10000 == 0:
            print("Read "+str(num)+" Written: "+str(rows))
        # print(line)
        text_tokens = tokenizer(line)['input_ids']
        # print(text_tokens[0:5])
        if len(text_tokens) > 1024: #Recipe won't fit the model
            continue

        # text_tokens_ids = tokenizer.convert_tokens_to_ids(text_tokens)
        text_tokens_ids = text_tokens
        # print(text_tokens_ids[0:5])

        if (len(last) + len(text_tokens_ids)) <= 1024:
            last+=text_tokens_ids
        else:
            while len(last) < 1024:
                last.append(end_token_id)
            out_np.append(last)
            last=text_tokens_ids
            rows+=1
    out_mat = np.matrix(out_np)
    print(out_mat.shape)
    # print(out_mat)
    hf.create_dataset(filename, data=out_mat)
hf.close()



(27, 1024)


Token indices sequence length is longer than the specified maximum sequence length for this model (1125 > 1024). Running this sequence through the model will result in indexing errors


Read 10000 Written: 2624
Read 20000 Written: 5275
Read 30000 Written: 7887
Read 40000 Written: 10514
Read 50000 Written: 13131
Read 60000 Written: 15766
Read 70000 Written: 18401
Read 80000 Written: 21012
Read 90000 Written: 23630
Read 100000 Written: 26283
Read 110000 Written: 28891
Read 120000 Written: 31517
Read 130000 Written: 34168
Read 140000 Written: 36816
Read 150000 Written: 39422
Read 160000 Written: 42062
Read 170000 Written: 44710
Read 180000 Written: 47379
Read 190000 Written: 49976
Read 200000 Written: 52616
Read 210000 Written: 55257
Read 220000 Written: 57876
Read 230000 Written: 60518
Read 240000 Written: 63137
Read 250000 Written: 65748
Read 260000 Written: 68340
Read 270000 Written: 70973
Read 280000 Written: 73614
Read 290000 Written: 76211
Read 300000 Written: 78827
Read 310000 Written: 81477
Read 320000 Written: 84133
Read 330000 Written: 86782
Read 340000 Written: 89433
Read 350000 Written: 92058
(93222, 1024)


# fine-tune with GPT2

In [ ]:
# fine-tuning model with bart
# logging info
import logging
logger = logging.getLogger(__name__)

import glob
import logging
import random
import gc
import boto3
import shutil

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange
import logging
import math
import os
from dataclasses import dataclass, field
from typing import Optional
from torch.utils.data import Dataset
import h5py
import torch

from transformers import WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
# data loading
class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path='train', block_size=512):
        cached_features_file = "unsupervised.h5"

        logger.info("Loading features from cached file %s", cached_features_file)
        with h5py.File(cached_features_file, 'r') as f:
            if file_path=='test':
                self.examples = f[file_path][:] #this is a dev set, 10% of a test set
            else:
                self.examples = f[file_path][:]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item])

def load_and_cache_examples(args, tokenizer, evaluate=False):
    dataset = TextDataset(tokenizer, file_path="test" if evaluate else "train", block_size=args.block_size)
    print(dataset[0:5])
    return dataset

In [ ]:
parser = argparse.ArgumentParser()
## Required parameters
parser.add_argument("--train_data_file", default="unsupervised.h5", type=str, required=False,
                        help="The input training data file (a text file).")
parser.add_argument("--output_dir", default="./gdrive/MyDrive/COMP0087/model_checkpoint/350k_GPT2", type=str, required=False,
                        help="The output directory where the model predictions and checkpoints will be written.")

## Other parameters
parser.add_argument("--model_type", default="gpt2" ,type=str,   #"facebook/bart-base"
                        help="The model architecture to be fine-tuned.")
parser.add_argument("--model_name_or_path", default="gpt2", type=str,
                        help="The model checkpoint for weights initialization.")

parser.add_argument("--eval_data_file", default=None, type=str,
                        help="An optional input evaluation data file to evaluate the perplexity on (a text file).")

parser.add_argument("--config_name", default="", type=str,
                        help="Optional pretrained config name or path if not the same as model_name_or_path")
parser.add_argument("--tokenizer_name", default="", type=str,
                        help="Optional pretrained tokenizer name or path if not the same as model_name_or_path")
parser.add_argument("--cache_dir", default="", type=str,
                        help="Optional directory to store the pre-trained models downloaded from s3 (instread of the default one)")
parser.add_argument("--block_size", default=-1, type=int,
                        help="Optional input sequence length after tokenization."
                             "The training dataset will be truncated in block of this size for training."
                             "Default to the model max input length for single sentence inputs (take into account special tokens).")
parser.add_argument("--do_train", action='store_true',
                        help="Whether to run training.")
parser.add_argument("--do_eval", action='store_true',
                        help="Whether to run eval on the dev set.")
parser.add_argument("--evaluate_during_training", action='store_true',
                        help="Run evaluation during training at each logging step.")
parser.add_argument("--do_lower_case", action='store_true',
                        help="Set this flag if you are using an uncased model.")

parser.add_argument("--per_gpu_train_batch_size", default=3, type=int,
                        help="Batch size per GPU/CPU for training.")
parser.add_argument("--per_gpu_eval_batch_size", default=4, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
parser.add_argument("--learning_rate", default=5e-5, type=float,
                        help="The initial learning rate for Adam.")
parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                        help="Epsilon for Adam optimizer.")
parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
parser.add_argument("--num_train_epochs", default=1, type=float,
                        help="Total number of training epochs to perform.")
parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
parser.add_argument("--warmup_steps", default=0, type=int,
                        help="Linear warmup over warmup_steps.")

parser.add_argument('--logging_steps', type=int, default=50,
                        help="Log every X updates steps.")
parser.add_argument('--save_steps', type=int, default=500000,
                        help="Save checkpoint every X updates steps.")
parser.add_argument("--eval_all_checkpoints", action='store_true',
                        help="Evaluate all checkpoints starting with the same prefix as model_name_or_path ending and ending with step number")
parser.add_argument("--no_cuda", action='store_true',
                        help="Avoid using CUDA when available")
parser.add_argument('--overwrite_output_dir', action='store_true',
                        help="Overwrite the content of the output directory")
parser.add_argument('--overwrite_cache', action='store_true',
                        help="Overwrite the cached training and evaluation sets")
parser.add_argument("--aws_bucket", default="", type=str,
                        help="Whether to upload to specified bucket.")
args, unknown = parser.parse_known_args()

def setup_args_for_model(args):
  args.model_type ="gpt2" #"facebook/bart-base"
  args.model_type = ""
  
  

In [ ]:
def model_init(args,logger, model_class, tokenizer):
  if args.eval_data_file is None and args.do_eval:
    raise ValueError("Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
                         "or remove the --do_eval argument.")
  if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train and not args.overwrite_output_dir:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))
  device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
  args.n_gpu = torch.cuda.device_count()
  args.device = device

  # Setup logging
  logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt = '%m/%d/%Y %H:%M:%S',
                        level = logging.INFO)

  model = model_class.from_pretrained(args.model_name_or_path)
  model.resize_token_embeddings(len(tokenizer))

  if args.block_size <= 0:
    args.block_size = tokenizer.max_len_single_sentence  # Our input block size will be the max possible for the model
  args.block_size = min(args.block_size, tokenizer.max_len_single_sentence)
  model.to(args.device)

  logger.info("Training/evaluation parameters %s", args)
  return model, logger
  


In [ ]:
def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    
    model, optimizer = amp.initialize(model, optimizer, opt_level="O2")

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info("  Total train batch size = %d",
                   args.train_batch_size * args.gradient_accumulation_steps)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()

    
    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=False)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
        for step, batch in enumerate(epoch_iterator):
            logger.info(step)
            #inputs, labels = (batch[:,0:-1], batch[:,1:])
            inputs, labels = (batch, batch)
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            # print(inputs[,3:10],labels[,3:10])

            outputs = model(inputs,labels =labels)
            # print(outputs)
            loss = outputs['loss'] # model outputs are always tuple in transformers (see doc)
            print(loss)

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()

            tr_loss += loss.item()
            if (step) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar('loss', (tr_loss - logging_loss)/args.logging_steps, global_step)
                    logging_loss = tr_loss

                # if False:
                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    torch.save(args, os.path.join(output_dir, 'training_args.bin'))
                    tokenizer.save_pretrained(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)
                    if args.evaluate_during_training:  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar('eval_{}'.format(key), value, global_step)
                    if args.aws_bucket:
                        tgz = "checkpoint-{}.tar".format(global_step)
                        tardir(output_dir, tgz)
                        shutil.rmtree(output_dir)
                        s3 = boto3.resource('s3')
                        s3.Object(args.aws_bucket, "checkpoints-gpt-medium/"+tgz).upload_file(tgz)
                        os.remove(tgz)

            if args.max_steps > 0 and global_step > 2:
                epoch_iterator.close()
                break
            del inputs, labels, outputs, loss
            torch.cuda.empty_cache()
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    tb_writer.close()

    return global_step, tr_loss / global_step

In [ ]:
#model, logger = model_init(args,logger,BartForCausalLM,tokenizer)
model, logger = model_init(args,logger, GPT2LMHeadModel, tokenizer)
#model, logger = model_init(args,logger,BartForConditionalGeneration,BartTokenizer)

In [ ]:
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)
print(len(train_dataset.examples))
global_step, tr_loss = train(args, train_dataset, model, tokenizer)
print(" global_step = %.5f, average loss = %.5f" %(global_step, tr_loss))

## evaluation

In [ ]:
def save_model(args,model,tokenizer,model_class,tokenizer_class):
  # Create output directory if neede
  if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

  logger.info("Saving model checkpoint to %s", args.output_dir)
  # Save a trained model, configuration and tokenizer using `save_pretrained()`.
  # They can then be reloaded using `from_pretrained()`
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(args.output_dir)
  tokenizer.save_pretrained(args.output_dir)

  # Good practice: save your training arguments together with the trained model
  torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

  # Load a trained model and vocabulary that you have fine-tuned
  model = model_class.from_pretrained(args.output_dir)
  tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
  model.to(args.device)



In [ ]:
#model_class = BartForCausalLM
#tokenizer_class = BartTokenizer
#save_model(args,model,tokenizer,model_class,tokenizer_class)
model_class = GPT2LMHeadModel
tokenizer_class = GPT2Tokenizer
save_model(args,model,tokenizer,model_class,tokenizer_class)
#model_class = BartForConditionalGeneration
#tokenizer_class = BartTokenizer
#save_model(args,model,tokenizer,model_class,tokenizer_class)

In [ ]:
def evaluate(args, model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)

    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        batch = batch.to(args.device)

        with torch.no_grad():
            outputs = model(batch, labels=batch)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {
        "perplexity": perplexity
    }

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
# Evaluation
results = {}
checkpoints = [args.output_dir]
if args.eval_all_checkpoints:
  checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + '/**/' + WEIGHTS_NAME, recursive=True)))
  logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
logger.info("Evaluate the following checkpoints: %s", checkpoints)
for checkpoint in checkpoints:
  global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
  model_class = GPT2LMHeadModel#BartForConditionalGeneration#GPT2LMHeadModel # BartForCausalLM
  model = model_class.from_pretrained(checkpoint)
  model.to(args.device)
  result = evaluate(args, model, tokenizer, prefix=global_step)
  result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
  results.update(result)

# generation

In [ ]:
#model loading
#tokenizer = AutoTokenizer.from_pretrained("mbien/recipenlg")
#model = AutoModelWithLMHead.from_pretrained("mbien/recipenlg")

In [ ]:
#raw_text = 'tomato,egg'
#prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
#tokenizer.encode(prepared_input)[0:-1]

In [ ]:
#new_model = BartForCausalLM.from_pretrained("./gdrive/MyDrive/COMP0087/model_checkpoint")
#new_tokenizer = BartTokenizer.from_pretrained("./gdrive/MyDrive/COMP0087/model_checkpoint")
#new_model = GPT2LMHeadModel.from_pretrained(args.output_dir)
#new_tokenizer = GPT2LMHeadModel.from_pretrained(args.output_dir)
#new_model = BartForConditionalGeneration.from_pretrained(args.output_dir)
#new_tokenizer= BartForConditionalGeneration.from_pretrained(args.output_dir)

In [ ]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, context, tokenizer, num_samples=1, temperature=1, top_k=0, top_p=0.0, device='cpu'):
    end_token = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context
    with torch.no_grad():
        for _ in trange(length):
            inputs = {'input_ids': generated}
            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            # print(outputs)
            next_token_logits = outputs["logits"][0, -1, :] / temperature
            
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
            #print(next_token)
            if next_token.item() == end_token:
                break
    return generated

In [ ]:
#model = BartForCausalLM.from_pretrained('./gdrive/MyDrive/COMP0087/model_checkpoint/500k')
#tokenizer = BartTokenizer.from_pretrained('./gdrive/MyDrive/COMP0087/model_checkpoint/500k')
#tokenizer = AutoTokenizer.from_pretrained("mbien/recipenlg")
#model = AutoModelWithLMHead.from_pretrained("mbien/recipenlg")

model = GPT2LMHeadModel.from_pretrained('./gdrive/MyDrive/COMP0087/model_checkpoint/350k_GPT2')
tokenizer = GPT2Tokenizer.from_pretrained('./gdrive/MyDrive/COMP0087/model_checkpoint/350k_GPT2')
model.eval()
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50270, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [ ]:
# raw_text = args.prompt if args.prompt else input("Comma-separated ingredients, semicolon to close the list >>> ")
raw_text = 'noodle, cake'

prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
context_tokens = tokenizer.encode(prepared_input)[0:-1]
out = sample_sequence(
            model=model,
            context=context_tokens,
            tokenizer=tokenizer,
            length=1000,
            device = 'cuda' #args.device #'cpu' #
        )
out = out[0, len(context_tokens):].tolist()
text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
if "<RECIPE_END>" not in text:
  print(text)
  print("Failed to generate, recipe's too long")
  full_text = prepared_input + text
  print(full_text)
else:
  full_text = prepared_input + text
  print(full_text)
  markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
  recipe_n_title = markdown.split("<TITLE_START>")
  title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
  markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
  markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
  markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n").replace("<NEXT_INSTR>", "\n").replace("<INSTR_END>", "\n")
  #markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n\"").replace("<NEXT_INSTR>", "\n\"").replace("<INSTR_END>", "\n")
  markdown = re.sub("$ +#", "#", markdown)
  markdown = re.sub("( +`|` +)", "`", markdown)
  print(title+markdown)




  0%|          | 0/1000 [00:00<?, ?it/s]

  1%|          | 8/1000 [00:00<00:13, 75.62it/s]

  2%|▏         | 16/1000 [00:00<00:13, 74.91it/s]

  2%|▏         | 24/1000 [00:00<00:13, 73.94it/s]

  3%|▎         | 31/1000 [00:00<00:13, 71.93it/s]

  4%|▍         | 39/1000 [00:00<00:13, 71.61it/s]

  5%|▍         | 46/1000 [00:00<00:13, 70.64it/s]

  5%|▌         | 54/1000 [00:00<00:13, 71.67it/s]

  6%|▌         | 62/1000 [00:00<00:13, 71.65it/s]

  7%|▋         | 69/1000 [00:00<00:13, 70.41it/s]

  8%|▊         | 76/1000 [00:01<00:13, 69.03it/s]

  8%|▊         | 84/1000 [00:01<00:13, 69.22it/s]

  9%|▉         | 91/1000 [00:01<00:13, 68.35it/s]

 10%|▉         | 98/1000 [00:01<00:13, 67.90it/s]

 10%|█         | 105/1000 [00:01<00:13, 68.28it/s]

 11%|█▏        | 113/1000 [00:01<00:12, 69.03it/s]

 12%|█▏        | 120/1000 [00:01<00:12, 68.54it/s]

 13%|█▎        | 127/1000 [00:01<00:12, 67.57it/s]

 13%|█▎        | 134/1000 [00:01<00:13, 64.57it/s]

 14%|█▍        | 141/1000 [00:02<0

 <RECIPE_START> <INPUT_START> noodle <NEXT_INPUT>  cakepotatoes <NEXT_INPUT> green of onion <NEXT_INPUT> cocktail sauce <NEXT_INPUT> firm tofu <NEXT_INPUT> bitters <NEXT_INPUT> fresh ginger <NEXT_INPUT> salt <NEXT_INPUT> rice wine vinegar <NEXT_INPUT> fish sauce <NEXT_INPUT> unbleached white pepper <NEXT_INPUT> saffron thread <NEXT_INPUT> egg <NEXT_INPUT> Vegetable cooking spray <INPUT_END> <INGR_START> 1/2 cup dijon-style medium-grain rice noodles <NEXT_INGR> 2 large potatoes, peeled and cut in 1/2-inch cubes <NEXT_INGR> 1 cup thinly sliced green of onion <NEXT_INGR> 2 ounces cocktail sauce (such as German Kimchi(R)) <NEXT_INGR> 11 ounces firm tofu, chopped <NEXT_INGR> 2 12 slices firm tofu, halved <NEXT_INGR> 4 strips whole mange (about 12 cups) <NEXT_INGR> 1 tablespoon chopped fresh ginger <NEXT_INGR> 1 14 teaspoons salt <NEXT_INGR> 1 14 cups rice wine vinegar <NEXT_INGR> 1 tablespoon fish sauce <NEXT_INGR> 14 teaspoon unsalted white pepper <NEXT_INGR> 1 pinch pinch saffron thread <

# Evaluation


In [ ]:
# gold standard
#df_eval = gold_st
#df_eval.reset_index(drop=True, inplace=True)
df_eval = pd.read_csv("./gdrive/MyDrive/COMP0087/350k_GPT-2_Score.csv")
df_eval_ner = df_eval["NER"]  #NER
df_eval_dir = df_eval["directions"]

In [ ]:
def get_raw_input(NER):
    test_str = NER
    test_str = test_str.replace("[","")
    test_str = test_str.replace("]","")
    test_str = test_str.replace("\"","")

    return test_str

def get_instr(markdown):
  markdown = markdown.split("\n")
  if ' ## Instructions ##' in markdown:
    instr_index = markdown.index(' ## Instructions ##')
    
    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")

    if "" in output:
        output.remove("")    
    
    return output

  elif '## Instructions ##' in markdown:
    instr_index = markdown.index('## Instructions ##')

    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")
        
    if "" in output:
        output.remove("")    

    return output
    
  else:
    return ["failed to generate"]


def generate_recipe(raw_text):

    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    context_tokens = tokenizer.encode(prepared_input)[0:-1]
    out = sample_sequence(
                model=model,
                context=context_tokens,
                tokenizer=tokenizer,
                length=1000,
                device = 'cuda' #args.device # 'cpu' #
            )
    out = out[0, len(context_tokens):].tolist()
    text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
    if "<RECIPE_END>" not in text:
      print(text)
      print("Failed to generate, recipe's too long")
      full_text = prepared_input + text
      print(full_text)
      return generate_recipe(raw_text)
    else:
      full_text = prepared_input + text
      print(full_text)
      markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
      recipe_n_title = markdown.split("<TITLE_START>")
      if len(recipe_n_title)<=1:
        return generate_recipe(raw_text)
      title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
      markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
      markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
      #markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n\"").replace("<NEXT_INSTR>", "\n\"").replace("<INSTR_END>", "\n")
      markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n").replace("<NEXT_INSTR>", "\n").replace("<INSTR_END>", "\n")
      markdown = re.sub("$ +#", "#", markdown)
      markdown = re.sub("( +`|` +)", "`", markdown)
  
      return markdown

## directions v. testing

In [ ]:
directions =  [[] for i in range(10)]

for i in range(10):  #100
  raw_text = get_raw_input(df_eval_ner[i])
  
  for j in range(10):
    md = generate_recipe(raw_text)
    directions[i].append(get_instr(md))

## directions v. full

In [ ]:
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
directions =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)

  for j in range(replicated_size):
    print("recipe: ",j)
    md = generate_recipe(raw_text)
    directions[i].append(get_instr(md))

## cos_sim

In [ ]:
#func of cos_sim
import re
import math
from collections import Counter


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

In [ ]:
#cosine_similarity

def COS_SIM(df_eval_dir,directions):
  """
  df_eval_dir := gold_standard recipes
  directions  := corresponding recipes """

  avg = 0

  for i in range(len(directions)):
    best = 0
    for j in range(len(directions[i])):
      cos = get_result(" ".join(eval(df_eval_dir[i])),
                      " ".join(directions[i][j]))
      best = max(best, cos)

    avg += best

  avg = avg/len(directions)

  print("avg:", avg)

In [ ]:
COS_SIM(df_eval_dir,directions)

avg: 0.5421947290378641


## bleu/gleu

In [ ]:
pip install jiwer==2.2.0

     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149811 sha256=23e7e316b38362b3a5432ca96c49db6b25d28e46b48b9f22a28c3b05c2e2a78a
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
!pip install -U nltk

     |████████████████████████████████| 1.5MB 16.3MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import nltk
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction

import nltk.translate.gleu_score as gleu
import nltk.translate.meteor_score as meteor

In [ ]:
# Helper Func.
def list_to_words(recipe):
  words = []
  for i in recipe:
    words += i.split()

  return words

In [ ]:
# New BLEU/GLEU
def bleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        refs.append(list_to_words(i))

    smoothie = SmoothingFunction().method5
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie, weights=(1, 0, 0, 0))
    #score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie, weights=(.25, 0.25, 0.25, 0.25))
    return score_ref_a

def gleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        refs.append(list_to_words(i))

    score_ref_a = gleu.sentence_gleu(refs, hyp)
    return score_ref_a

In [ ]:
bleu_avg = []
gleu_avg = []

for i in range(len(directions)):
  # print(bleu_score(df_eval_dir[i], directions[i]))
  bleu_avg.append(bleu_score(df_eval_dir[i], directions[i]))

for i in range(len(directions)):
  gleu_avg.append(gleu_score(df_eval_dir[i], directions[i]))
print(np.mean(bleu_avg))
print(np.mean(gleu_avg))

0.7739315958814414
0.09064334028913239


## Old BLEU/GLEU

In [ ]:
def bleu_score(recipe, refer):
    hyp = recipe
    refs = refer
    smoothie = SmoothingFunction().method4
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie)
    return score_ref_a


def gleu_score(recipe, refer):
    hyp = recipe
    refs = refer
    score_ref_a = gleu.sentence_gleu([refs], hyp)
    return score_ref_a

In [ ]:
ret_bleu = []
ret_gleu = []
#ret_wer = []

for i in range(100):
  
  for j in range(10):

    res_bleu = bleu_score(get_raw_input(df_eval_dir[i]),get_raw_input(directions[i][j]))

    res_gleu = gleu_score(get_raw_input(df_eval_dir[i]),get_raw_input(directions[i][j]))

    #res_wer = wer_score(get_raw_input(df_eval_dir[i]),get_raw_input(directions[i][j]))

    ret_bleu.append(res_bleu)
    ret_gleu.append(res_gleu)
    #ret_wer.append(res_wer)

print(np.mean(ret_bleu))
print(np.mean(ret_gleu))
#print(np.mean(ret_wer))


## Save Results

In [ ]:
df_eval["GPT2-350K"] = directions
df_eval.to_csv("./gdrive/MyDrive/COMP0087/350k_GPT-2_Score.csv")

#cccc


In [ ]:

"""
def wer(ref, hyp ,debug=False):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY=1 # Tact
    INS_PENALTY=1 # Tact
    SUB_PENALTY=1 # Tact
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("Ncor " + str(numCor))
        print("Nsub " + str(numSub))
        print("Ndel " + str(numDel))
        print("Nins " + str(numIns))
    return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'Cor':numCor, 'Sub':numSub, 'Ins':numIns, 'Del':numDel}

"""

In [ ]:
'''
def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
    d = d.reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion    = d[i][j-1] + 1
                deletion     = d[i-1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]

if __name__ == "__main__":
    import doctest
    doctest.testmod()

'''